In [ ]:
# cosmus_eval_major_label.py
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from transformers import pipeline

# -------------------------------------------------------------------- #
# 1.  Load the COSMUS dataset (Telegram RU/UA posts)                   #
# -------------------------------------------------------------------- #
ds = load_dataset("YShynkarov/COSMUS", split="train") 
ds = ds.filter(lambda x: x["annotator_sentiment"] != "mixed")
df = ds.to_pandas()[["document_content", "annotator_sentiment", "language"]]


Filter:   0%|          | 0/12224 [00:00<?, ? examples/s]

In [44]:
df

,document_content,annotator_sentiment,language
0,⚡️Українська делегація відправилася на перемов...,neutral,ua
1,"Вибухи на Одещині, попередньо — ППО.",neutral,ua
2,"А что делать тем ,кто лишился своего жилья ,по...",negative,ru
3,Тогда учись быстро бегать. Для меня вопрос сло...,negative,ru
4,Добрий день,neutral,ua
...,...,...,...
11611,"У меня три окна и двери выбило , даже и не дум...",negative,ru
11612,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",negative,ua
11613,"Питання, цей сертифікат можна вже використовув...",neutral,ua
11614,На Вугледарському напрямку загинув Рома Іванен...,negative,ua


In [31]:
df['document_content'].iloc[5]

'Бажаю удачі тим, хто цього потребує.'

'Я розумію. Але ви хоч уявляєте, скільки часу на це піде? І не буде там великої суми, бо рахуватимуть тільки вартість "коробки". Опис майна "до" ніхто не робив.'

In [45]:

# Gold labels → integers
label2id = {"negative": -1, "neutral": 0, "positive": 1}
df["annotator_sentiment"] = df["annotator_sentiment"].map(label2id)


In [33]:
df = df.dropna()
df

,document_content,annotator_sentiment,language
0,⚡️Українська делегація відправилася на перемов...,0.0,ua
1,"Вибухи на Одещині, попередньо — ППО.",0.0,ua
2,"А что делать тем ,кто лишился своего жилья ,по...",-1.0,ru
3,Тогда учись быстро бегать. Для меня вопрос сло...,-1.0,ru
4,Добрий день,0.0,ua
...,...,...,...
12218,"У меня три окна и двери выбило , даже и не дум...",-1.0,ru
12219,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",-1.0,ua
12221,"Питання, цей сертифікат можна вже використовув...",0.0,ua
12222,На Вугледарському напрямку загинув Рома Іванен...,-1.0,ua


In [ ]:
# -------------------------------------------------------------------- #
# 2.  Sentiment model wrapper with major_label()                       #
# -------------------------------------------------------------------- #
class SentimentAnalyzer:
    """
    Multilingual twitter-XLM-RoBERTa sentiment wrapper.
    Provides polarity_scores() *and* major_label().
    """
    def __init__(self):
        mdl = "cardiffnlp/twitter-xlm-roberta-base-sentiment"       # 3-way (neg/neu/pos)
        self._pipe = pipeline(
            "sentiment-analysis",
            model=mdl, tokenizer=mdl,
            top_k=None                                              # returns all three scores  
        )

    # ---------- already supplied ----------
    def polarity_scores(self, text: str):
        res   = self._pipe(text)              # list[list[dict(label,score)]]
        scores = {d["label"]: d["score"] for d in res[0]}
        # compound = abs(scores.get("positive", 0) - scores.get("negative", 0))
        return {"neg": scores.get("negative", 0),
                "neu": scores.get("neutral",  0),
                "pos": scores.get("positive", 0),
                # "compound": compound,
                }

    # ---------- new method ----------
    def major_label(self, text: str):
        """
        Returns (text_label, int_label) where int_label ∈ {−1,0,1}.
        """
        sc   = self.polarity_scores(text)
        best = max(("neg", "neu", "pos"), key=sc.get)           
        text_label = {"neg": "negative", "neu": "neutral", "pos": "positive"}[best]
        return text_label, {"negative": -1, "neutral": 0, "positive": 1}[text_label]

analyzer = SentimentAnalyzer()

Device set to use mps:0


In [ ]:
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer, pipeline
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

class UkrSentimentAnalyzer:
    """
    Ukrainian sentiment analysis model based on YShynkarov/ukr-roberta-cosmus-sentiment.
    Provides polarity_scores() and major_label().
    """
    map_labels = {
                'LABEL_0': 'mixed',
                'LABEL_1': 'negative',
                'LABEL_2': 'neutral',
                'LABEL_3': 'positive',
            }
    int_label_map = {
        "negative": -1.0,
        "neutral": 0.0,
        "positive": 1.0,
        "mixed": 0
    }

    def __init__(self):
        repo_id = "YShynkarov/ukr-roberta-cosmus-sentiment"
        safetensor = hf_hub_download(repo_id=repo_id,
                                     filename="ukrroberta_cosmus_sentiment.safetensors")

        config = RobertaConfig.from_pretrained("youscan/ukr-roberta-base", num_labels=4)
        tokenizer = RobertaTokenizer.from_pretrained("youscan/ukr-roberta-base")

        model = RobertaForSequenceClassification(config)
        state_dict = load_file(safetensor)
        model.load_state_dict(state_dict)
        model.eval()
        self._pipe = pipeline(
            "text-classification",
            model=model,
            tokenizer=tokenizer,
            device=-1,               
            return_all_scores=True,
            truncation=True,
        )

    def polarity_scores(self, text: str) -> dict:
        """
        Returns dict:
          {
            "negative": float_score,
            "neutral":  float_score,
            "positive": float_score,
            "mixed":    float_score
          }
        """
        # pipeline returns list[list[{"label":..., "score":...}, ...]]
        results = self._pipe(text)
        scores = {
            self.map_labels[item["label"]]: item["score"]
            for item in results[0]
        }
        return scores

    def major_label(self, text: str) -> tuple[str, int]:
        """
        Returns (text_label, int_label),
        where int_label ∈ {-1,0,1,2} for negative, neutral, positive, mixed.
        """
        scores = self.polarity_scores(text)
        best = max(scores, key=scores.get)
        return best, self.int_label_map[best]

analyzer = UkrSentimentAnalyzer()
print(analyzer.polarity_scores("Привіт! Все просто чудово"))
# → {'negative': 0.01, 'neutral': 0.05, 'positive': 0.90, 'mixed': 0.04}

print(analyzer.major_label("Привіт! Все просто чудово"))
# → ('positive', 1)

Device set to use cpu


{'mixed': 0.011259634047746658, 'negative': 0.0018209181725978851, 'neutral': 0.006787061225622892, 'positive': 0.9801324009895325}
('positive', 1.0)


/Users/pelmeshek1706/Desktop/projects/phd_311_venv/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [54]:
# -------------------------------------------------------------------- #
# 3.  Inference → sentiment_pred column                                #
# -------------------------------------------------------------------- #
from tqdm import tqdm      # или просто `from tqdm import tqdm`
tqdm.pandas()

df["sentiment_pred_1epoch"] = df["document_content"].progress_apply(
    lambda txt: analyzer.major_label(txt)[1]     # keep numeric only
)

100%|██████████| 11616/11616 [12:04<00:00, 16.04it/s]


In [55]:
df=df.dropna(subset=["annotator_sentiment", 'sentiment_pred_1epoch'])
df



,document_content,annotator_sentiment,language,sentiment_pred_1epoch
0,⚡️Українська делегація відправилася на перемов...,0,ua,1.0
1,"Вибухи на Одещині, попередньо — ППО.",0,ua,0.0
2,"А что делать тем ,кто лишился своего жилья ,по...",-1,ru,-1.0
3,Тогда учись быстро бегать. Для меня вопрос сло...,-1,ru,-1.0
4,Добрий день,0,ua,1.0
...,...,...,...,...
11611,"У меня три окна и двери выбило , даже и не дум...",-1,ru,-1.0
11612,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",-1,ua,1.0
11613,"Питання, цей сертифікат можна вже використовув...",0,ua,0.0
11614,На Вугледарському напрямку загинув Рома Іванен...,-1,ua,1.0


In [56]:
df["sentiment_pred_1epoch"].value_counts()

sentiment_pred_1epoch
 0.0    5764
-1.0    3345
 1.0    2507
Name: count, dtype: int64

In [ ]:
# -------------------------------------------------------------------- #
# 4.  Evaluation                                                       #
# -------------------------------------------------------------------- #
acc = accuracy_score(df["annotator_sentiment"], df["sentiment_pred_1epoch"])
print(f"Accuracy: {acc:.3%}")

print(classification_report(
      df["annotator_sentiment"], df["sentiment_pred_1epoch"],
      target_names=["negative (−1)", "neutral (0)", "positive (+1)"]))


Accuracy: 76.799%
               precision    recall  f1-score   support

negative (−1)       0.90      0.66      0.76      4541
  neutral (0)       0.71      0.87      0.78      4702
positive (+1)       0.73      0.77      0.75      2373

     accuracy                           0.77     11616
    macro avg       0.78      0.77      0.76     11616
 weighted avg       0.79      0.77      0.77     11616



# Ukr -> Eng -> Sentiment by Vader (default sentiment in Openwillis)

In [ ]:
# uk2en_like_space.py
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "Yehor/kulyk-uk-en"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16

REVISION = None

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, revision=REVISION)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map=device,
    torch_dtype=torch_dtype,
    revision=REVISION,
)
model.eval()

def translate_like_space(text: str) -> str:
    prompt = "Translate the text to English:\n" + text 
    input_ids = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        add_generation_prompt=True,
        return_tensors="pt",
        tokenize=True,
    ).to(model.device)

    with torch.inference_mode():
        output = model.generate(
            input_ids,
            max_new_tokens=2048,
            do_sample=False,              # greedy
            repetition_penalty=1.05,     
        )

    gen = output[:, input_ids.shape[1]:]
    return tokenizer.batch_decode(gen, skip_special_tokens=True)[0].strip()


In [ ]:

uk = "Над Україною збито ракету та 7 із 8 «Шахедів»"
print(translate_like_space(uk))

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

df['translated_text'] = df['document_content'].progress_apply(translate_like_space)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def vader_label_by_max(text: str) -> int:
    scores = analyzer.polarity_scores(text)
    top = max(('neg', 'neu', 'pos'), key=lambda k: scores[k])
    return {'neg': -1.0, 'neu': 0.0, 'pos': 1.0}[top]

In [ ]:

df['vader_analysis'] = df['translated_text'].apply(vader_label_by_max)


In [ ]:
df['vader_analysis'].value_counts()

In [ ]:
df["annotator_sentiment"].value_counts()

In [ ]:
# -------------------------------------------------------------------- #
# 4.  Evaluation                                                       #
# -------------------------------------------------------------------- #
acc = accuracy_score(df["annotator_sentiment"], df["vader_analysis"])
print(f"Accuracy: {acc:.3%}")

print(classification_report(
      df["annotator_sentiment"], df["vader_analysis"],
      target_names=["negative (−1)", "neutral (0)", "positive (+1)"]))
